In [1]:
import torch 
import lightning as L
import yaml
import sys, os
sys.path.append('../')
from lightning_scripts.lightning_classifier import LitWordAudioSetModel 
import importlib
from pathlib import Path 
from lightning_scripts.jsinV3DataLoader_precombined_batched import jsinV3_precombined_all_signals



In [2]:
# get config and init trained model 
config_path = Path("lightning_scripts/configs/word_audioset_resnet50.yaml")
config = yaml.load(open(config_path, 'r'), Loader=yaml.FullLoader)

config['num_workers'] = 4
config['hparas']['batch_size'] = 96 # set to single-gpu size 

# update val set to use entire range 
config['data']['eval_max'] = -1

checkpoint_dir = Path("model_checkpoints") / f"{config_path.stem}/checkpoints"
ckpt_paths = sorted(checkpoint_dir.glob("*.ckpt"), key=os.path.getctime)
ckpt_path = ckpt_paths[-1] # get latest checkpoint 
print(ckpt_path)

model_checkpoints/word_audioset_resnet50/checkpoints/epoch=5-step=42750-best_word_task.ckpt


In [3]:
model = LitWordAudioSetModel.load_from_checkpoint(checkpoint_path=ckpt_path, config=config)

In [4]:
config['data']

{'root': '/mnt/ceph/users/jfeather/data/training_datasets_audio/JSIN_all_v3/subsets/',
 'target_keys': ['signal/word_int', 'noise/labels_binary_via_int'],
 'eval_max': -1}

In [5]:
## init val dataloader

val_loader = model.val_dataloader() # will be populated with relevant args in config 

In [6]:
trainer = L.Trainer(devices=1)


/mnt/home/igriffith/ceph/conda_envs/cochdnn_ssl_pl/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/home/igriffith/ceph/conda_envs/cochdnn_ssl_pl/l ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [7]:
trainer.validate(model, dataloaders=val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined